# Feature Extraction Module

In [1]:
import csv
import pywt
import pyedflib
import numpy as np
from spectrum import *
from os import listdir
from nitime import utils
import scipy.stats as sp
from os.path import isfile, join
from nitime.viz import plot_tseries
from matplotlib import pyplot as plt
from nitime import algorithms as alg
from nitime.timeseries import TimeSeries

## Features List!

In [2]:
names = ['Fractal Dimension','Coeffiecient of Variation','Mean of Vertex to Vertex Slope','Variance of Vertex to Vertex Slope',
         'Hjorth_Activity','Hjorth_Mobility','Hjorth_Complexity',
         'Kurtosis','2nd Difference Mean','2nd Difference Max',
         'Skewness','1st Difference Mean','1st Difference Max',
         'FFT Delta MaxPower','FFT Theta MaxPower','FFT Alpha MaxPower','FFT Beta MaxPower','Delta/Theta','Delta/Alpha','Theta/Alpha','(Delta+Theta)/Alpha',
         '1Wavelet Approximate Mean','Wavelet Approximate Std Deviation','Wavelet Approximate Energy','Wavelet Detailed Mean','Wavelet Detailed Std Deviation','Wavelet Detailed Energy','Wavelet Approximate Entropy','Wavelet Detailed Entropy',
         '2Wavelet Approximate Mean','Wavelet Approximate Std Deviation','Wavelet Approximate Energy','Wavelet Detailed Mean','Wavelet Detailed Std Deviation','Wavelet Detailed Energy','Wavelet Approximate Entropy','Wavelet Detailed Entropy',
         '3Wavelet Approximate Mean','Wavelet Approximate Std Deviation','Wavelet Approximate Energy','Wavelet Detailed Mean','Wavelet Detailed Std Deviation','Wavelet Detailed Energy','Wavelet Approximate Entropy','Wavelet Detailed Entropy',
         '4Wavelet Approximate Mean','Wavelet Approximate Std Deviation','Wavelet Approximate Energy','Wavelet Detailed Mean','Wavelet Detailed Std Deviation','Wavelet Detailed Energy','Wavelet Approximate Entropy','Wavelet Detailed Entropy',
         '5Wavelet Approximate Mean','Wavelet Approximate Std Deviation','Wavelet Approximate Energy','Wavelet Detailed Mean','Wavelet Detailed Std Deviation','Wavelet Detailed Energy','Wavelet Approximate Entropy','Wavelet Detailed Entropy',
         '6Wavelet Approximate Mean','Wavelet Approximate Std Deviation','Wavelet Approximate Energy','Wavelet Detailed Mean','Wavelet Detailed Std Deviation','Wavelet Detailed Energy','Wavelet Approximate Entropy','Wavelet Detailed Entropy',
         '7Wavelet Approximate Mean','Wavelet Approximate Std Deviation','Wavelet Approximate Energy','Wavelet Detailed Mean','Wavelet Detailed Std Deviation','Wavelet Detailed Energy','Wavelet Approximate Entropy','Wavelet Detailed Entropy',
         '8Wavelet Approximate Mean','Wavelet Approximate Std Deviation','Wavelet Approximate Energy','Wavelet Detailed Mean','Wavelet Detailed Std Deviation','Wavelet Detailed Energy','Wavelet Approximate Entropy','Wavelet Detailed Entropy',
         '9Wavelet Approximate Mean','Wavelet Approximate Std Deviation','Wavelet Approximate Energy','Wavelet Detailed Mean','Wavelet Detailed Std Deviation','Wavelet Detailed Energy','Wavelet Approximate Entropy','Wavelet Detailed Entropy',
         '10Wavelet Approximate Mean','Wavelet Approximate Std Deviation','Wavelet Approximate Energy','Wavelet Detailed Mean','Wavelet Detailed Std Deviation','Wavelet Detailed Energy','Wavelet Approximate Entropy','Wavelet Detailed Entropy',
         '11Wavelet Approximate Mean','Wavelet Approximate Std Deviation','Wavelet Approximate Energy','Wavelet Detailed Mean','Wavelet Detailed Std Deviation','Wavelet Detailed Energy','Wavelet Approximate Entropy','Wavelet Detailed Entropy',
         '12Wavelet Approximate Mean','Wavelet Approximate Std Deviation','Wavelet Approximate Energy','Wavelet Detailed Mean','Wavelet Detailed Std Deviation','Wavelet Detailed Energy','Wavelet Approximate Entropy','Wavelet Detailed Entropy',
         '13Wavelet Approximate Mean','Wavelet Approximate Std Deviation','Wavelet Approximate Energy','Wavelet Detailed Mean','Wavelet Detailed Std Deviation','Wavelet Detailed Energy','Wavelet Approximate Entropy','Wavelet Detailed Entropy',
         '14Wavelet Approximate Mean','Wavelet Approximate Std Deviation','Wavelet Approximate Energy','Wavelet Detailed Mean','Wavelet Detailed Std Deviation','Wavelet Detailed Energy','Wavelet Approximate Entropy','Wavelet Detailed Entropy',
         'AR1','AR2','AR3','AR4','AR5','AR6','AR7','AR8','AR9','AR10','AR11','AR12','AR13','AR14','AR15','AR16','AR17','AR18',
         'AR19','AR20','AR21','AR22','AR23','AR24','AR25','AR26','AR27','AR28','AR29','AR30','AR31','AR32','AR33','AR34','AR35','AR36','AR37','AR38','AR39','AR40','AR41','AR42'
        ]

### Fractal Dimension!

In [3]:
def fractal_dimension(Z, threshold=0.9):

    # Only for 2d image
    assert(len(Z.shape) == 2)

    # From https://github.com/rougier/numpy-100 (#87)
    def boxcount(Z, k):
        S = np.add.reduceat(
            np.add.reduceat(Z, np.arange(0, Z.shape[0], k), axis=0),
                               np.arange(0, Z.shape[1], k), axis=1)

        # We count non-empty (0) and non-full boxes (k*k)
        return len(np.where((S > 0) & (S < k*k))[0])


    # Transform Z into a binary array
    Z = (Z < threshold)

    # Minimal dimension of image
    p = min(Z.shape)

    # Greatest power of 2 less than or equal to p
    n = 2**np.floor(np.log(p)/np.log(2))

    # Extract the exponent
    n = int(np.log(n)/np.log(2))

    # Build successive box sizes (from 2**n down to 2**1)
    sizes = 2**np.arange(n, 1, -1)

    # Actual box counting with decreasing size
    counts = []
    for size in sizes:
        counts.append(boxcount(Z, size))

    # Fit the successive log(sizes) with log (counts)
    coeffs = np.polyfit(np.log(sizes), np.log(counts), 1)
    return -coeffs[0]

### Coefficient of Varaition

In [4]:
def coeff_var(a):
    b = a #Extracting the data from the 14 channels
    output = np.zeros(len(b)) #Initializing the output array with zeros
    k = 0; #For counting the current row no.
    for i in b:
        mean_i = np.mean(i) #Saving the mean of array i
        std_i = np.std(i) #Saving the standard deviation of array i
        output[k] = std_i/mean_i #computing coefficient of variation
        k=k+1
    return np.sum(output)/14

### Mean of Vertex to Vertex Slope

In [5]:
import heapq
from scipy.signal import argrelextrema


def first_diff(i):
    b=i    
    
    out = np.zeros(len(b))
    
    for j in range(len(i)):
        out[j] = b[j-1]-b[j]# Obtaining the 1st Diffs
        
        j=j+1
        c=out[1:len(out)]
    return c #returns first diff


def slope_mean(p):
    b = p #Extracting the data from the 14 channels
    output = np.zeros(len(b)) #Initializing the output array with zeros
    res = np.zeros(len(b)-1)
    
    k = 0; #For counting the current row no.
    for i in b:
        x=i
        amp_max = i[argrelextrema(x, np.greater)[0]]
        t_max = argrelextrema(x, np.greater)[0]
        amp_min = i[argrelextrema(x, np.less)[0]]
        t_min = argrelextrema(x, np.less)[0]
        t = np.concatenate((t_max,t_min),axis=0)
        t.sort()#sort on the basis of time

        h=0
        amp = np.zeros(len(t))
        res = np.zeros(len(t)-1)
        for l in range(len(t)):
            amp[l]=i[t[l]]
           
        
        amp_diff = first_diff(amp)
        
        t_diff = first_diff(t)
        
        for q in range(len(amp_diff)):
            res[q] = amp_diff[q]/t_diff[q]         
        output[k] = np.mean(res) 
        k=k+1
    return np.sum(output)/14


### Variance of Vertex to Vertex Slope

In [6]:
import heapq
from scipy.signal import argrelextrema


def first_diff(i):
    b=i    
    
    out = np.zeros(len(b))
    
    for j in range(len(i)):
        out[j] = b[j-1]-b[j]# Obtaining the 1st Diffs
        
        j=j+1
        c=out[1:len(out)]
    return c #returns first diff


def slope_var(p):
    b = p #Extracting the data from the 14 channels
    output = np.zeros(len(b)) #Initializing the output array with zeros
    res = np.zeros(len(b)-1)
    
    k = 0; #For counting the current row no.
    for i in b:
        x=i
        amp_max = i[argrelextrema(x, np.greater)[0]]#storing maxima value
        t_max = argrelextrema(x, np.greater)[0]#storing time for maxima
        amp_min = i[argrelextrema(x, np.less)[0]]#storing minima value
        t_min = argrelextrema(x, np.less)[0]#storing time for minima value
        t = np.concatenate((t_max,t_min),axis=0) #making a single matrix of all matrix
        t.sort() #sorting according to time

        h=0
        amp = np.zeros(len(t))
        res = np.zeros(len(t)-1)
        for l in range(len(t)):
            amp[l]=i[t[l]]
           
        
        amp_diff = first_diff(amp)
        
        t_diff = first_diff(t)
        
        for q in range(len(amp_diff)):
            res[q] = amp_diff[q]/t_diff[q] #calculating slope        
    
        output[k] = np.var(res) 
        k=k+1#counting k
    return np.sum(output)/14

### Hjorth    

In [7]:
def hjorth(input):                                             # function for hjorth 
    realinput = input
    hjorth_activity = np.zeros(len(realinput))
    hjorth_mobility = np.zeros(len(realinput))
    hjorth_diffmobility = np.zeros(len(realinput))
    hjorth_complexity = np.zeros(len(realinput))
    diff_input = np.diff(realinput)
    diff_diffinput = np.diff(diff_input)
    k = 0
    for j in realinput:
        hjorth_activity[k] = np.var(j)
        hjorth_mobility[k] = np.sqrt(np.var(diff_input[k])/hjorth_activity[k])
        hjorth_diffmobility[k] = np.sqrt(np.var(diff_diffinput[k])/np.var(diff_input[k]))
        hjorth_complexity[k] = hjorth_diffmobility[k]/hjorth_mobility[k]
        k = k+1
    return np.sum(hjorth_activity)/14, np.sum(hjorth_mobility)/14, np.sum(hjorth_complexity)/14                       #returning hjorth activity, hjorth mobility , hjorth complexity

### Kurtosis

In [8]:
def kurtosis(a):
    b = a # Extracting the data from the 14 channels
    output = np.zeros(len(b)) # Initializing the output array with zeros (length = 14)
    k = 0; # For counting the current row no.
    for i in b:
        mean_i = np.mean(i) # Saving the mean of array i
        std_i = np.std(i) # Saving the standard deviation of array i
        t = 0.0
        for j in i:
            t += (pow((j-mean_i)/std_i,4)-3)
        kurtosis_i = t/len(i) # Formula: (1/N)*(summation(x_i-mean)/standard_deviation)^4-3
        output[k] = kurtosis_i # Saving the kurtosis in the array created
        k +=1 # Updating the current row no.
    return np.sum(output)/14

### Second Difference Mean

In [9]:
def secDiffMean(a):
    b = a # Extracting the data of the 14 channels
    output = np.zeros(len(b)) # Initializing the output array with zeros (length = 14)
    temp1 = np.zeros(len(b[0])-1) # To store the 1st Diffs
    k = 0; # For counting the current row no.
    for i in b:
        t = 0.0
        for j in range(len(i)-1):
            temp1[j] = abs(i[j+1]-i[j]) # Obtaining the 1st Diffs
        for j in range(len(i)-2):
            t += abs(temp1[j+1]-temp1[j]) # Summing the 2nd Diffs
        output[k] = t/(len(i)-2) # Calculating the mean of the 2nd Diffs
        k +=1 # Updating the current row no.
    return np.sum(output)/14

### Second Difference Max

In [10]:
def secDiffMax(a):
    b = a # Extracting the data from the 14 channels
    output = np.zeros(len(b)) # Initializing the output array with zeros (length = 14)
    temp1 = np.zeros(len(b[0])-1) # To store the 1st Diffs
    k = 0; # For counting the current row no.
    t = 0.0
    for i in b:
        for j in range(len(i)-1):
            temp1[j] = abs(i[j+1]-i[j]) # Obtaining the 1st Diffs
        t = temp1[1] - temp1[0]
        for j in range(len(i)-2):
            if abs(temp1[j+1]-temp1[j]) > t :
                t = temp1[j+1]-temp1[j] # Comparing current Diff with the last updated Diff Max

        output[k] = t # Storing the 2nd Diff Max for channel k
        k +=1 # Updating the current row no.
    return np.sum(output)/14


### Skewness

In [11]:
def skewness(arr):
    data = arr 
    skew_array = np.zeros(len(data)) #Initialinling the array as all 0s
    index = 0; #current cell position in the output array
   
    for i in data:
        skew_array[index]=sp.stats.skew(i,axis=0,bias=True)
        index+=1 #updating the cell position
    return np.sum(skew_array)/14

### First Difference Mean

In [12]:
def first_diff_mean(arr):
    data = arr 
    diff_mean_array = np.zeros(len(data)) #Initialinling the array as all 0s
    index = 0; #current cell position in the output array
   
    for i in data:
        sum=0.0#initializing the sum at the start of each iteration
        for j in range(len(i)-1):
            sum += abs(i[j+1]-i[j]) # Obtaining the 1st Diffs
           
        diff_mean_array[index]=sum/(len(i)-1)
        index+=1 #updating the cell position
    return np.sum(diff_mean_array)/14

### First Difference Max

In [13]:
def first_diff_max(arr):
    data = arr 
    diff_max_array = np.zeros(len(data)) #Initialinling the array as all 0s
    first_diff = np.zeros(len(data[0])-1)#Initialinling the array as all 0s 
    index = 0; #current cell position in the output array
   
    for i in data:
        max=0.0#initializing at the start of each iteration
        for j in range(len(i)-1):
            first_diff[j] = abs(i[j+1]-i[j]) # Obtaining the 1st Diffs
            if first_diff[j]>max: 
                max=first_diff[j] # finding the maximum of the first differences
        diff_max_array[index]=max
        index+=1 #updating the cell position
    return np.sum(diff_max_array)/14

## Wavelet Fetures!
### Approx Mean, Approx Std Deviation, Approx Energy, Detailed Mean, Detailed Std Deviation, Detailed Energy, Approx Entropy & Detailed Entropy

In [14]:
def wavelet_features(epoch,channels):
    cA_values = []
    cD_values = []
    cA_mean = []
    cA_std = []
    cA_Energy =[]
    cD_mean = []
    cD_std = []
    cD_Energy = []
    Entropy_D = []
    Entropy_A = []
    wfeatures = []
    for i in range(channels):
        cA,cD=pywt.dwt(epoch[i,:],'coif1')
        cA_values.append(cA)
        cD_values.append(cD)		#calculating the coefficients of wavelet transform.
    for x in range(channels):   
        cA_mean.append(np.mean(cA_values[x]))
        wfeatures.append(np.mean(cA_values[x]))
        
        cA_std.append(abs(np.std(cA_values[x])))
        wfeatures.append(abs(np.std(cA_values[x])))
        
        cA_Energy.append(abs(np.sum(np.square(cA_values[x]))))
        wfeatures.append(abs(np.sum(np.square(cA_values[x]))))
        
        cD_mean.append(np.mean(cD_values[x]))		# mean and standard deviation values of coefficents of each channel is stored .
        wfeatures.append(np.mean(cD_values[x]))

        cD_std.append(abs(np.std(cD_values[x])))
        wfeatures.append(abs(np.std(cD_values[x])))
        
        cD_Energy.append(abs(np.sum(np.square(cD_values[x]))))
        wfeatures.append(abs(np.sum(np.square(cD_values[x]))))
        
        Entropy_D.append(abs(np.sum(np.square(cD_values[x]) * np.log(np.square(cD_values[x])))))
        wfeatures.append(abs(np.sum(np.square(cD_values[x]) * np.log(np.square(cD_values[x])))))
        
        Entropy_A.append(abs(np.sum(np.square(cA_values[x]) * np.log(np.square(cA_values[x]))))) 
        wfeatures.append(abs(np.sum(np.square(cA_values[x]) * np.log(np.square(cA_values[x])))))
    return wfeatures

### FFT Max Power - Delta, Theta, Alpha & Beta Band!

In [15]:
from scipy import signal

def maxPwelch(data_win,Fs):
 
    
    BandF = [0.1, 3, 7, 12, 30]
    PMax = np.zeros([14,(len(BandF)-1)]);
    
    for j in range(14):
        f,Psd = signal.welch(data_win[j,:], Fs)
        
        for i in range(len(BandF)-1):
            fr = np.where((f>BandF[i]) & (f<=BandF[i+1]))
            PMax[j,i] = np.max(Psd[fr])
    
    return np.sum(PMax[:,0])/14,np.sum(PMax[:,1])/14,np.sum(PMax[:,2])/14,np.sum(PMax[:,3])/14


## Shanon Entropy

In [16]:
def shanon_entropy(labels): # Shanon Entropy
    """ Computes entropy of 0-1 vector. """
    n_labels = len(labels)
    counts = np.bincount(labels)
    probs = counts[np.nonzero(counts)] / n_labels
    n_classes = len(probs)

    if n_classes <= 1:
        return 0
    return - np.sum(probs * np.log(probs)) / np.log(n_classes)



##  Spectral Entropy

In [17]:
from numpy.fft import fft
from numpy import zeros, floor, log10, log, mean, array, sqrt, vstack, cumsum, ones, log2, std
from numpy.linalg import svd, lstsq
import time

def bin_power(X,Band,Fs):
    C = fft(X)
    C = abs(C)
    Power =zeros(len(Band)-1);
    for Freq_Index in xrange(0,len(Band)-1):
        Freq = float(Band[Freq_Index])   ## Xin Liu
        Next_Freq = float(Band[Freq_Index+1])
        Power[Freq_Index] = sum(C[floor(Freq/Fs*len(X)):floor(Next_Freq/Fs*len(X))])
    Power_Ratio = Power/sum(Power)
    return Power, Power_Ratio


def spectral_entropy(X, Fs, Power_Ratio = None):
    
    Band = [0.1, 3, 7, 12, 30]
    if Power_Ratio is None:
        Power, Power_Ratio = bin_power(X, Band, Fs)

    Spectral_Entropy = 0
    for i in xrange(0, len(Power_Ratio) - 1):
        Spectral_Entropy += Power_Ratio[i] * log(Power_Ratio[i])
    Spectral_Entropy /= log(len(Power_Ratio))     # to save time, minus one is omitted
    print('Shape of Spectral Entropy = ',n.shape(Spectral_Entropy))
    return -1 * Spectral_Entropy


## Autoregressive model

### Autoregression - Yule Walker Algorithm

In [18]:
def autogressiveModelParameters(labels):
    b_labels = len(labels)
    feature = []
    for i in range(14):
        coeff, sig = alg.AR_est_YW(labels[i,:], 11,)
        feature.append(coeff)
    a = []     
    for i in range(11):
        a.append(np.sum(feature[:][i])/14)
     
    return a

### Autoregression  - Burg Algorithm

In [19]:
def autogressiveModelParametersBurg(labels):
    feature = []
    feature1 = []
    model_order = 3
    for i in range(14):
        AR, rho, ref = arburg(labels[i], model_order)
        feature.append(AR);
    for j in range(14):
        for i in range(model_order):
            feature1.append(feature[j][i])

    return feature1

## Feature Vector Calculation

In [20]:
idlefiles  = [f for f in listdir('Training-Data/Idle') if isfile(join('Training-Data/Idle', f))] 
OneBackfiles = [f for f in listdir('Training-Data/1-Back') if isfile(join('Training-Data/1-Back', f))]
#TwoBackfiles = [f for f in listdir('Training-Data/2-Back') if isfile(join('Training-Data/2-Back', f))]
#DualOneBackfiles = [f for f in listdir('Training-Data/Dual-1-Back') if isfile(join('Training-Data/Dual-1-Back', f))]
#DualTwoBackfiles = [f for f in listdir('Training-Data/Dual-2-Back') if isfile(join('Training-Data/Dual-2-Back', f))]

files = []

for i in idlefiles:
    files.append([i,'Idle'])
    
for i in OneBackfiles:
    files.append([i,'1-Back'])

#for i in TwoBackfiles:
#    files.append([i,'2-Back'])

#for i in DualOneBackfiles:
#    files.append([i,'Dual-1-Back'])

#for i in DualTwoBackfiles:
#    files.append([i,'Dual-2-Back'])
    
    
mypath = 'Training-Data/'
csvfile = 'Features/features.csv'

with open(csvfile, "a") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerow(names)
    for counter in range(len(files)):
        subfolder =  files[counter][1]
        tag = files[counter][1] 
        #print(tag)
        data_path = mypath + subfolder +'/'+files[counter][0]
        
        print(data_path)
        rows = csv.reader(open(data_path))
        sigbufs = [l for l in rows]
        sigbufs = np.array(sigbufs)
        sigbufs = sigbufs.transpose()
        sigbufs = sigbufs.astype(float)

        for i in np.arange(1,150,3):
            features = []
            epoch = sigbufs[:,i*128:(i+3)*128]
            if len(epoch[0]) == 0:
                break
            
            # Fractal Dimension
            features.append(fractal_dimension(epoch))
            
            #Coeffeicient of Variation
            features.append(coeff_var(epoch))

            #Mean of Vertex to Vertex Slope
            features.append(slope_mean(epoch))
            
            #Variance of Vertex to Vertex Slope
            features.append(slope_var(epoch))
       
            #Hjorth Parameters
            feature_list = hjorth(epoch)
            for feat in feature_list:
                features.append(feat)
            
            #Kurtosis
            features.append(kurtosis(epoch))

            #Second Difference Mean
            features.append(secDiffMean(epoch))

            #Second Difference Max
            features.append(secDiffMax(epoch))
               
            #Skewness
            features.append(skewness(epoch))
 
            #First Difference Mean
            features.append(first_diff_mean(epoch))

            #First Difference Max
            features.append(first_diff_max(epoch))
            
            #FFT Max Power - Delta, Theta, Alpha & Beta Band!
            feature_list  =  maxPwelch(epoch,128)            
            for feat in feature_list:
                features.append(feat)
            #FFT Frequency Ratios
            features.append(feature_list[0]/feature_list[1])
            features.append(feature_list[0]/feature_list[2])
            features.append(feature_list[1]/feature_list[3])
            features.append((feature_list[0] + feature_list[1])/feature_list[2])
            
            #Wavelet Fetures!      
            feature_list = wavelet_features(epoch,14)
            for feat in feature_list:
                features.append(feat)
                    
            #Autoregressive model Coefficients
            feature_list = autogressiveModelParametersBurg(epoch)
            for feat in feature_list:
                features.append(feat.real)
       
            """   
            #Shanon Entropy
            abc = shanon_entropy(epoch)
            print('Shape SE = ',np.shape(abc))
            features.append(abc)
            
            #Spectral Entropy
            feature_list = spectral_entropy(epoch,128)
            for feat in feature_list:
                features.append(feat)        
            """
            
            tag
            if tag == 'Idle':
                myClass = 1
            
            if tag == '1-Back':
                myClass = 2
            
#            if tag == '2-Back':
#               myClass = 3

#            if tag == 'Dual-1-Back':
#                myClass = 4
            
#            if tag == 'Dual-2-Back':
#                myClass = 5
            
            features.append(myClass);        
            writer.writerow(features) 
        
            
            

Training-Data/Idle/Clean_EEG_Data_S07_5.csv
Training-Data/Idle/Clean_EEG_Data_S4_1.csv
Training-Data/Idle/Clean_EEG_Data_S10_5.csv
Training-Data/Idle/Clean_EEG_Data_S2_1.csv
Training-Data/Idle/Clean_EEG_Data_S3_1.csv
Training-Data/Idle/Clean_EEG_Data_S09_5.csv
Training-Data/Idle/Clean_EEG_Data_S5_1.csv
Training-Data/Idle/Clean_EEG_Data_S1_1.csv
Training-Data/Idle/Clean_EEG_Data_S08_5.csv
Training-Data/Idle/Clean_EEG_Data_S11_5.csv
Training-Data/1-Back/Clean_EEG_Data_S11_1.csv
Training-Data/1-Back/Clean_EEG_Data_S08_1.csv


/home/baljeet/.local/lib/python2.7/site-packages/scipy/signal/spectral.py:772: UserWarning: nperseg = 256, is greater than input length = 98, using nperseg = 98
  'using nperseg = {1:d}'.format(nperseg, x.shape[-1]))


Training-Data/1-Back/Clean_EEG_Data_S4_1.csv
Training-Data/1-Back/Clean_EEG_Data_S07_1.csv
Training-Data/1-Back/Clean_EEG_Data_S2_1.csv
Training-Data/1-Back/Clean_EEG_Data_S5_1.csv
Training-Data/1-Back/Clean_EEG_Data_S09_1.csv
Training-Data/1-Back/Clean_EEG_Data_S1_1.csv
Training-Data/1-Back/Clean_EEG_Data_S10_1.csv
Training-Data/1-Back/Clean_EEG_Data_S3_1 .csv


## Normalisation

In [21]:
r = csv.reader(open('Features/features.csv')) 
lines = [l for l in r]

In [22]:
for i in range(len(lines[1])-1):  
    columns = []
    for j in range(1,len(lines)):
        columns.append(float(lines[j][i]))
    mean = np.mean(columns,axis = 0)
    #print('\nMean = ',mean)
    std_dev  = np.std(columns,axis = 0)
    #print('\nSTD Deviation = ',std_dev)
    for j in range(1,len(lines)):
        lines[j][i] = (float(lines[j][i])-mean)/std_dev

writer = csv.writer(open('Features/Normalizedfeatures.csv', 'wb'))
writer.writerows(lines)
print('Done!')

Done!
